In [21]:
import numpy as np
import pandas as pd
import time
import ast
import json
import datetime
import pprint


import warnings 
warnings.filterwarnings('ignore')

import logging
from kiteconnect import KiteConnect

from kiteconnect import KiteConnect, KiteTicker
import time
import threading

logging.basicConfig(level=logging.ERROR)

from kiteconnect import KiteConnect
import os
import pickle

pd.options.display.max_rows = 500
pd.options.display.max_columns = 1000

In [ ]:
kite = KiteConnect(api_key=api_key)
data = kite.generate_session(request_token, api_secret=api_secret)
access_token = data["access_token"]
kite.set_access_token(access_token)

In [26]:
instruments = kite.instruments()
inst_list = [
             "ICICIBANK","HDFCBANK","YESBANK","AXISBANK",\
             "INFY","TCS",\
             "TATAMOTORS","TVSMOTOR"\
            ]



In [1]:
dict_inst ={}
inst_dict = {} 
inst_ticks = {}
for inst in instruments:
    if inst.get("instrument_type") == 'EQ' and  inst.get("exchange") == "NSE":
        symbol = inst.get("tradingsymbol")
        dict_inst[symbol] = inst.get("instrument_token")
        inst_dict[inst.get("instrument_token")] = symbol
        inst_ticks[symbol] = inst.get("tick_size")
        if symbol in inst_list:
            print(inst)

In [28]:
print('last 5 days')
returns_dict ={}
delta_dict = {}
vol_dict = {}
for symbol in  inst_list:
    instrument_token = dict_inst.get(symbol)
    interval = "5minute"

    to_date = datetime.date.today()
    from_date =  to_date - datetime.timedelta(days=5)

    data = kite.historical_data(
        instrument_token=instrument_token,
        from_date=from_date,
        to_date=to_date,
        interval=interval,
        continuous=False,
        oi=False
    )

    close_prices = []
    print("--------------",symbol)
    
    df = pd.DataFrame(data)
    df['ticker'] =symbol
    df.to_csv(f'{symbol}.csv',index=False)
    


last 5 days
-------------- ICICIBANK
-------------- HDFCBANK
-------------- YESBANK
-------------- AXISBANK
-------------- INFY
-------------- TCS
-------------- TATAMOTORS
-------------- TVSMOTOR


## Plan of Action

- For each ticker, the opening range is constructed
- Based on opening bullish/bearish chandle, the breakout level is determined. Scope to use retracement levels
- Based on Fibonacci Golden ratio Extension , target is determined


In [ ]:
dfs= {}
for symbol in  inst_list:
    df = pd.read_csv(f'{symbol}.csv')

    df['dt'] = pd.to_datetime(df.date).dt.date

    bullish_df= df.groupby('dt').first().apply(lambda row:row['open']-row['close'] > 0,axis=1).to_frame().reset_index()

    bullish_df = bullish_df.rename({0:'bullish'},axis=1)

    OR_df = df.groupby('dt').first().apply(lambda row:row['high']-row['low'],axis=1).to_frame().reset_index()

    OR_df = OR_df.rename({0:'OR'},axis=1)
    
    
    df = df.merge(bullish_df).merge(OR_df)

    ORB_Target_df = df.groupby('dt').first().apply(lambda row: row['low'] + 1.618 *  row['OR'] if row['bullish'] else row['high'] - 1.618 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_Target_df = ORB_Target_df.rename({0:'ORB_golden_ratio_target'},axis=1)

    ORB_breakout_df = df.groupby('dt').first().apply(lambda row: row['high'] if row['bullish'] else row['low'],axis=1).to_frame().reset_index()

    ORB_breakout_df= ORB_breakout_df.rename({0:'ORB_breakout_level'},axis=1)

    df = df.merge(ORB_Target_df).merge(ORB_breakout_df)
    dfs[symbol]=df

In [ ]:
dfs.get(inst_list[0]).head(0)